In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

In [ ]:
file_name = "NF-CSE-CIC-IDS2018-v2.csv"
data = pd.read_csv(file_name)

In [ ]:
data.Label.value_counts()

0    16635567
1     2258141
Name: Label, dtype: int64

In [ ]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [ ]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [ ]:
data.Attack.unique()

array(['SSH-Bruteforce', 'Benign', 'DDoS attacks-LOIC-HTTP',
       'DDOS attack-HOIC', 'DoS attacks-Slowloris', 'DoS attacks-Hulk',
       'FTP-BruteForce', 'Infilteration', 'Bot', 'DoS attacks-GoldenEye',
       'Brute Force -Web', 'DoS attacks-SlowHTTPTest', 'SQL Injection',
       'DDOS attack-LOIC-UDP', 'Brute Force -XSS'], dtype=object)

In [ ]:
data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)

In [ ]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,1663557,1663557,1663557,1663557,1663557,1663557,1663557,1663557,1663557,1663557,...,1663557,1663557,1663557,1663557,1663557,1663557,1663557,1663557,1663557,1663557
Bot,14310,14310,14310,14310,14310,14310,14310,14310,14310,14310,...,14310,14310,14310,14310,14310,14310,14310,14310,14310,14310
Brute Force -Web,214,214,214,214,214,214,214,214,214,214,...,214,214,214,214,214,214,214,214,214,214
Brute Force -XSS,93,93,93,93,93,93,93,93,93,93,...,93,93,93,93,93,93,93,93,93,93
DDOS attack-HOIC,108086,108086,108086,108086,108086,108086,108086,108086,108086,108086,...,108086,108086,108086,108086,108086,108086,108086,108086,108086,108086
DDOS attack-LOIC-UDP,211,211,211,211,211,211,211,211,211,211,...,211,211,211,211,211,211,211,211,211,211
DDoS attacks-LOIC-HTTP,30730,30730,30730,30730,30730,30730,30730,30730,30730,30730,...,30730,30730,30730,30730,30730,30730,30730,30730,30730,30730
DoS attacks-GoldenEye,2772,2772,2772,2772,2772,2772,2772,2772,2772,2772,...,2772,2772,2772,2772,2772,2772,2772,2772,2772,2772
DoS attacks-Hulk,43265,43265,43265,43265,43265,43265,43265,43265,43265,43265,...,43265,43265,43265,43265,43265,43265,43265,43265,43265,43265


In [ ]:
X = data.drop(columns=["Attack", "Label"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)

In [ ]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [ ]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [ ]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [ ]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h
10660988,172.31.66.109,172.31.0.2,1.401126e-08,1.108397e-08,0.000078,1.237078e-06,0.000098,1.237078e-06,1.405050e-08,1.405050e-08,...,0.0,0.0,0.0,1.458031e-07,1.458029e-07,1.347021e-10,1.109630e-08,0.000074,1.478535e-07,"[1.4011259652278748e-08, 1.108397479973663e-08..."
15147714,172.31.66.68,172.31.0.2,1.245158e-08,1.008341e-08,0.000079,1.099371e-06,0.000097,1.099371e-06,1.248645e-08,1.248645e-08,...,0.0,0.0,0.0,1.295728e-07,1.295727e-07,4.406332e-11,9.861101e-09,0.000066,1.313950e-07,"[1.2451577640895502e-08, 1.0083409899455216e-0..."
9587672,172.31.64.28,172.31.0.2,1.552131e-08,1.227854e-08,0.000077,1.370403e-06,0.000099,1.370403e-06,1.556478e-08,1.556478e-08,...,0.0,0.0,0.0,1.615169e-07,1.615167e-07,7.435637e-12,1.229219e-08,0.000082,1.637882e-07,"[1.5521310572476826e-08, 1.2278540225056721e-0..."
7746586,172.31.67.18,172.31.0.2,7.552795e-09,5.974837e-09,0.000045,6.668492e-07,0.000086,6.668492e-07,7.573948e-09,7.573948e-09,...,0.0,0.0,0.0,7.859541e-08,7.859535e-08,1.331115e-12,5.883835e-09,0.000040,7.970069e-08,"[7.55279530717907e-09, 5.974837018934749e-09, ..."
693958,172.31.67.31,172.31.0.2,3.657431e-09,2.893307e-09,0.000049,6.458418e-07,0.000115,6.458418e-07,3.667675e-09,3.667675e-09,...,0.0,0.0,0.0,3.805972e-08,3.805969e-08,4.762408e-12,2.896524e-09,0.000010,3.859495e-08,"[3.6574312874776492e-09, 2.893307042209973e-09..."


In [ ]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [ ]:
# Training graph

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [ ]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [ ]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [ ]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [ ]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [ ]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [ ]:
# Convert to GPU
train_g = train_g

In [ ]:
# cnt_wait = 0
# best = 1e9
# best_t = 0
# dur = []
# node_features = train_g.ndata['h']
# edge_features = train_g.edata['h']

# for epoch in range(epochs):
#     dgi.train()
#     if epoch >= 3:
#         t0 = time.time()

#     dgi_optimizer.zero_grad()
#     loss = dgi(train_g, node_features, edge_features)
#     loss.backward()
#     dgi_optimizer.step()

#     if loss < best:
#         best = loss
#         best_t = epoch
#         cnt_wait = 0
#         torch.save(dgi.state_dict(), 'best_dgi.pkl')
#     else:
#         cnt_wait += 1

#   # if cnt_wait == patience:
#   #     print('Early stopping!')
#   #     break

#     if epoch >= 3:
#         dur.append(time.time() - t0)

#     if epoch % 50 == 0:

#         print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
#             "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
#               loss.item(),
#               train_g.num_edges() / np.mean(dur) / 1000))

In [ ]:
dgi.load_state_dict(torch.load('best_dgi.pkl'))

<All keys matched successfully>

In [ ]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [ ]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [ ]:
# Convert to GPU
test_g = test_g

In [ ]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [ ]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [ ]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,0.106326,0.002315,-0.000980,-0.022264,0.045113,0.029210,0.038371,-0.002847,-0.040849,0.057440,...,0.046839,0.001730,0.012890,-0.099413,0.043458,0.072058,0.035648,0.047931,Benign,0
1,0.106326,0.002315,-0.000980,-0.022264,0.045113,0.029210,0.038371,-0.002847,-0.040849,0.057440,...,0.046839,0.001730,0.012890,-0.099413,0.043458,0.072058,0.035648,0.047931,Benign,0
2,0.106326,0.002315,-0.000980,-0.022264,0.045113,0.029210,0.038371,-0.002847,-0.040849,0.057440,...,0.046839,0.001730,0.012890,-0.099413,0.043458,0.072058,0.035648,0.047931,Benign,0
3,0.106326,0.002315,-0.000980,-0.022264,0.045113,0.029210,0.038371,-0.002847,-0.040849,0.057440,...,0.046839,0.001730,0.012890,-0.099413,0.043458,0.072058,0.035648,0.047931,Benign,0
4,0.106326,0.002315,-0.000980,-0.022264,0.045113,0.029210,0.038371,-0.002847,-0.040849,0.057440,...,0.046839,0.001730,0.012890,-0.099413,0.043458,0.072058,0.035648,0.047931,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641549,0.123069,0.026578,-0.009467,0.007449,0.056632,0.034634,0.045075,-0.007112,-0.050270,0.071016,...,0.054894,-0.003832,-0.008389,-0.097075,0.057076,0.087696,0.034345,0.045828,Benign,0
2641550,0.146425,0.022692,-0.004799,0.044749,0.055015,0.036507,0.058755,-0.027695,-0.030468,0.092988,...,0.026060,-0.017071,-0.058810,-0.093998,0.085114,0.140869,0.067376,0.048734,Benign,0
2641551,0.140820,0.066262,-0.041707,0.046161,0.078733,0.042034,0.057938,-0.020710,-0.051066,0.092359,...,0.037817,-0.008337,-0.050818,-0.079956,0.083160,0.120430,0.043901,0.045791,Benign,0
2641552,0.147100,0.046359,-0.023473,0.045124,0.067559,0.038634,0.062342,-0.026802,-0.040660,0.093196,...,0.026881,-0.011808,-0.052880,-0.089512,0.086857,0.129162,0.059013,0.048242,Benign,0


# Embeddings CBLOF  Embeddings

In [ ]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [ ]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [ ]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [ ]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [5:30:26<00:00, 550.74s/it]  


{'n_estimators': 5, 'con': 0.001}
0.944374438967276
              precision    recall  f1-score   support

           0     0.9770    0.9989    0.9878    996643
           1     0.9899    0.8267    0.9010    135488

    accuracy                         0.9783   1132131
   macro avg     0.9834    0.9128    0.9444   1132131
weighted avg     0.9785    0.9783    0.9774   1132131



In [ ]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [5:34:38<00:00, 557.74s/it]  


{'n_estimators': 2, 'con': 0.1}
0.9437829594960284
              precision    recall  f1-score   support

           0     0.9769    0.9986    0.9876    996643
           1     0.9874    0.8267    0.8999    135488

    accuracy                         0.9780   1132131
   macro avg     0.9822    0.9126    0.9438   1132131
weighted avg     0.9782    0.9780    0.9771   1132131



In [ ]:
###  CBLOF RAW

In [ ]:
df_raw_train = pd.concat([X_train.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_train], axis=1)
df_raw_test = pd.concat([X_test.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_test], axis=1)

In [ ]:
raw_benign_train_samples = df_raw_train[df_raw_train.Label == 0].drop(columns=["Label", "Attack"])
raw_normal_train_samples = df_raw_train.drop(columns=["Label", "Attack"])

raw_train_labels = df_raw_train["Label"]
raw_test_labels = df_raw_test["Label"]

raw_test_samples = df_raw_test.drop(columns=["Label", "Attack"])

In [ ]:
from pyod.models.cblof import CBLOF

n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = CBLOF(n_clusters=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  3%|▎         | 1/36 [00:13<07:58, 13.68s/it]

2


  6%|▌         | 2/36 [00:27<07:56, 14.02s/it]

2


  8%|▊         | 3/36 [00:47<09:03, 16.48s/it]

...

2


 17%|█▋        | 6/36 [01:34<07:52, 15.74s/it]

2


100%|██████████| 36/36 [20:22<00:00, 33.97s/it]


{'n_estimators': 9, 'con': 0.04}
0.8741086740526474
              precision    recall  f1-score   support

           0     0.9762    0.9600    0.9681    499068
           1     0.7377    0.8278    0.7801     67744

    accuracy                         0.9442    566812
   macro avg     0.8570    0.8939    0.8741    566812
weighted avg     0.9477    0.9442    0.9456    566812



In [ ]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [20:42<00:00, 34.51s/it]


benign only
{'n_estimators': 2}
0.8618432811826696
              precision    recall  f1-score   support

           0     0.9591    0.9806    0.9697    499068
           1     0.8287    0.6916    0.7540     67744

    accuracy                         0.9461    566812
   macro avg     0.8939    0.8361    0.8618    566812
weighted avg     0.9435    0.9461    0.9439    566812



In [ ]:
# HBOS  Embeddings

In [ ]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [ ]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [ ]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [19:29<00:00, 32.49s/it]


{'n_estimators': 5, 'con': 0.01}
0.945069359337394
              precision    recall  f1-score   support

           0     0.9845    0.9897    0.9871    996643
           1     0.9213    0.8855    0.9030    135488

    accuracy                         0.9772   1132131
   macro avg     0.9529    0.9376    0.9451   1132131
weighted avg     0.9769    0.9772    0.9770   1132131



In [ ]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [21:10<00:00, 35.28s/it]


{'n_estimators': 5, 'con': 0.1}
0.9189314026948445
              precision    recall  f1-score   support

           0     0.9705    0.9945    0.9824    996643
           1     0.9503    0.7779    0.8555    135488

    accuracy                         0.9686   1132131
   macro avg     0.9604    0.8862    0.9189   1132131
weighted avg     0.9681    0.9686    0.9672   1132131



In [ ]:
##  HBOS  RAw

In [ ]:
from pyod.models.cblof import CBLOF

n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = HBOS(n_bins=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [02:09<00:00,  3.59s/it]


{'n_estimators': 30, 'con': 0.04}
0.8627757960209628
              precision    recall  f1-score   support

           0     0.9715    0.9601    0.9658    499068
           1     0.7294    0.7928    0.7598     67744

    accuracy                         0.9401    566812
   macro avg     0.8505    0.8765    0.8628    566812
weighted avg     0.9426    0.9401    0.9412    566812



In [ ]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = HBOS(n_bins=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [02:14<00:00,  3.74s/it]


benign only
{'n_estimators': 5}
0.7882018992795334
              precision    recall  f1-score   support

           0     0.9766    0.8943    0.9337    499068
           1     0.5197    0.8422    0.6427     67744

    accuracy                         0.8881    566812
   macro avg     0.7481    0.8683    0.7882    566812
weighted avg     0.9220    0.8881    0.8989    566812



In [ ]:
##  PCA  Emb

In [ ]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [29:47<00:00, 49.66s/it]


{'n_estimators': 10, 'con': 0.001}
0.9442956641768174
              precision    recall  f1-score   support

           0     0.9770    0.9988    0.9878    996643
           1     0.9896    0.8267    0.9008    135488

    accuracy                         0.9782   1132131
   macro avg     0.9833    0.9127    0.9443   1132131
weighted avg     0.9785    0.9782    0.9774   1132131



In [ ]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [32:26<00:00, 54.07s/it]


{'n_estimators': 10, 'con': 0.1}
0.9256946497974641
              precision    recall  f1-score   support

           0     0.9723    0.9955    0.9838    996643
           1     0.9598    0.7916    0.8676    135488

    accuracy                         0.9711   1132131
   macro avg     0.9661    0.8935    0.9257   1132131
weighted avg     0.9708    0.9711    0.9699   1132131



In [ ]:
##  PCA  RAw

In [ ]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples)
   
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [07:28<00:00, 12.47s/it]


{'n_estimators': 20, 'con': 0.1}
0.7684270275042566
              precision    recall  f1-score   support

           0     0.9639    0.9009    0.9313    499068
           1     0.5071    0.7513    0.6055     67744

    accuracy                         0.8830    566812
   macro avg     0.7355    0.8261    0.7684    566812
weighted avg     0.9093    0.8830    0.8924    566812



In [ ]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples)

    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [08:04<00:00, 13.46s/it]


benign only
{'n_estimators': 10}
0.7376147683157477
              precision    recall  f1-score   support

           0     0.9622    0.8744    0.9162    499068
           1     0.4466    0.7471    0.5590     67744

    accuracy                         0.8591    566812
   macro avg     0.7044    0.8107    0.7376    566812
weighted avg     0.9006    0.8591    0.8735    566812



In [ ]:
##  IF  Emb

In [ ]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [3:09:47<00:00, 474.46s/it]  


{'n_estimators': 20, 'con': 0.001}
0.9538863735449246
              precision    recall  f1-score   support

           0     0.9805    0.9992    0.9897    996643
           1     0.9928    0.8538    0.9180    135488

    accuracy                         0.9818   1132131
   macro avg     0.9866    0.9265    0.9539   1132131
weighted avg     0.9820    0.9818    0.9812   1132131



In [ ]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [3:31:56<00:00, 529.86s/it]  


{'n_estimators': 50, 'con': 0.2}
0.8110535459623227
              precision    recall  f1-score   support

           0     0.9878    0.8952    0.9392    996643
           1     0.5436    0.9184    0.6829    135488

    accuracy                         0.8979   1132131
   macro avg     0.7657    0.9068    0.8111   1132131
weighted avg     0.9346    0.8979    0.9085   1132131



In [ ]:
##  IF  Raw

In [ ]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples.to_numpy())
   
    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [19:12<00:00, 48.03s/it]


{'n_estimators': 20, 'con': 0.05}
0.8176793439704795
              precision    recall  f1-score   support

           0     0.9599    0.9494    0.9547    499068
           1     0.6553    0.7082    0.6807     67744

    accuracy                         0.9206    566812
   macro avg     0.8076    0.8288    0.8177    566812
weighted avg     0.9235    0.9206    0.9219    566812



In [ ]:
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples.to_numpy())

    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [20:38<00:00, 51.60s/it]


benign only
{'n_estimators': 100}
0.7409370706714709
              precision    recall  f1-score   support

           0     0.9633    0.8755    0.9173    499068
           1     0.4512    0.7539    0.5646     67744

    accuracy                         0.8610    566812
   macro avg     0.7072    0.8147    0.7409    566812
weighted avg     0.9021    0.8610    0.8751    566812

